In [41]:
from cdp_data import CDPInstances, datasets

boston_df = datasets.get_session_dataset(
    CDPInstances.Boston,
    start_datetime="2023-01-26",
    end_datetime="2023-01-27",
    store_transcript=True,
    store_transcript_as_csv=True,
)
boston_df.sample(1)

Converting transcripts: 100%|██████████| 1/1 [00:00<00:00,  1.15it/s]


,session_datetime,session_index,session_content_hash,video_uri,video_start_time,video_end_time,caption_uri,external_source_id,id,key,event,transcript,transcript_path,transcript_as_csv_path
0,2023-01-26 19:00:00+00:00,0,56647e4d969bad6ce521ff23601c5d2bf1703db48761d9...,https://storage.googleapis.com/download/storag...,None,None,http://boston.granicus.com//videos/993/caption...,None,8aefa4d65844,session/8aefa4d65844,<cdp_backend.database.models.Event object at 0...,<cdp_backend.database.models.Transcript object...,/Users/shakragoler/VS_Code/cdp/minutes-item-se...,/Users/shakragoler/VS_Code/cdp/minutes-item-se...


In [42]:
from functools import partial
import pandas as pd

def _process_session(row: pd.Series, extra_meta: dict) -> dict:
    # Read transcript CSV
    transcript = pd.read_csv(row.transcript_as_csv_path)
    
    return {
        "text": " ".join(transcript.text),
        "meta": {
            "event_id": row.event.id,
            "session_id": row.id,
            **extra_meta,
        },
    }

# Create partial with muni name
process_session_boston = partial(_process_session, extra_meta={"muni": "boston"})

# Process all rows in dataset to get full text to a single column
processed_sessions = pd.DataFrame(
    list(boston_df.apply(process_session_boston, axis=1)), 
)
processed_sessions.sample(1)

,text,meta
0,You You You You You You You You You You You Yo...,"{'event_id': '1561c0756c10', 'session_id': '8a..."


In [43]:
# Store processed dataset to JSONL
processed_sessions.to_json("boston.jsonl", orient="records", lines=True)